In [31]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import importlib
from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy


In [32]:
PATH='data/language_model/'

TRN_PATH = 'train/all/'
VAL_PATH = 'test/all/'
TRN = f'{PATH}{TRN_PATH}' 
VAL = f'{PATH}{VAL_PATH}' 

In [33]:
spacy_tok = spacy.load('en')

In [46]:
TEXT = data.Field(lower=True, tokenize="spacy")

In [55]:
bs=8; bptt=70

In [56]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [57]:
% rm data/language_model/models/TEXT.pkl
% mkdir data/language_model/models
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

mkdir: cannot create directory ‘data/language_model/models’: File exists


The # batches; # unique tokens in the vocab; # tokens in the training set; # sentences

In [58]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(85932, 57669, 1, 48122517)

In [59]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', '-', ':', '*', '_', '<', '%', '>', ',', '"', '/']

In [60]:
TEXT.vocab.stoi['attorney']

4162

In [61]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers
opt_fn = partial(optim.Adam, betas=(0.7, 0.99)) # Adam with less momentum

In [62]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss                                  
    0      2.608248   2.624434  
    1      2.134051   2.741595                                  
    2      2.48611    2.511693                                  
    3      2.374393   2.721114                                  
    4      2.218454   2.682373                                  
    5      2.111241   2.650925                                  
    6      2.797106   2.494725                                  
    7      2.093039   2.777062                                  
    8      2.354067   2.732445                                  
    9      2.121959   2.745906                                  
    10     2.166346   2.696924                                  
    11     2.106757   2.660498                                  
    12     2.420785   2.634942                                  
    13     2.27844    2.677652                                  
    14     2.733559   2.454241                           

[array([2.45424])]

In [65]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))


In [63]:
learner.save_encoder('adam1_enc')

In [ ]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

In [ ]:
learner.save_encoder('adam3_10_enc')

In [ ]:
learner.save_encoder('adam3_20_enc')

In [ ]:
learner.load_encoder('adam3_20_enc')
